# Tabular models

In [1]:
from fastai import *
from fastai.tabular import *
from fastai.callbacks import EarlyStoppingCallback, ReduceLROnPlateauCallback
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import StratifiedKFold

Tabular data should be in a Pandas `DataFrame`.

In [2]:
path = Path('../Data/cat-in-the-dat-ii/') #untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'train.csv')
df.set_index('id',drop=True,inplace=True)
test_df = pd.read_csv(path/'test.csv')
test_df.set_index('id',drop=True,inplace=True)
sample = pd.read_csv(path/'sample_submission.csv')

In [3]:
path

PosixPath('../Data/cat-in-the-dat-ii')

In [4]:
df.tail()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
599995,0.0,1.0,0.0,T,N,Red,Polygon,Axolotl,India,Theremin,...,015c63324,3.0,Novice,Freezing,a,R,GZ,5.0,NaN,0
599996,1.0,0.0,0.0,T,Y,Blue,Polygon,Dog,Costa Rica,Oboe,...,a02ae6a63,2.0,Novice,Boiling Hot,n,N,sf,NaN,3.0,0
599997,0.0,0.0,0.0,F,Y,Red,Circle,Axolotl,Russia,Theremin,...,40f9610c1,2.0,Contributor,Freezing,n,H,MV,7.0,5.0,0
599998,1.0,1.0,0.0,F,Y,NaN,Polygon,Axolotl,NaN,Piano,...,c1a8374a0,1.0,Master,Warm,m,X,Ey,1.0,5.0,0
599999,0.0,0.0,0.0,T,N,Blue,Triangle,Dog,Russia,Theremin,...,e2aea7784,1.0,Contributor,Boiling Hot,b,O,uI,5.0,8.0,0


In [5]:
test_df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,...,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,...,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,...,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,...,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,Theremin,...,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0


## Do some EDA

In [6]:
# df.id.nunique()

In [7]:
df.bin_0.nunique(), df.bin_1.nunique(), df.bin_2.nunique(), df.bin_3.nunique(), df.bin_4.nunique()

(2, 2, 2, 2, 2)

In [8]:
df.nom_0.nunique(), df.nom_1.nunique(), df.nom_2.nunique(), df.nom_3.nunique(), df.nom_4.nunique(), df.nom_5.nunique(), df.nom_6.nunique(), df.nom_7.nunique(), df.nom_8.nunique(), df.nom_9.nunique(), 

(3, 6, 6, 6, 4, 1220, 1519, 222, 222, 2218)

In [9]:
df.ord_0.nunique(), df.ord_1.nunique(), df.ord_2.nunique(), df.ord_3.nunique(), df.ord_4.nunique(), df.ord_5.nunique() 

(3, 5, 6, 15, 26, 190)

In [10]:
df.day.nunique(), df.month.nunique()

(7, 12)

In [11]:
df.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2',
       'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0',
       'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month', 'target'],
      dtype='object')

In [12]:
dep_var = 'target'
cat_names = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
             'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8',
             'nom_9','ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5',
             'day', 'month']
# cont_names = ['id']
procs = [FillMissing, Categorify, Normalize]

In [13]:
df.bin_0.dtype

dtype('float64')

In [14]:
embed = {}
# for col in cat_names:
# #     print(col)
#     if df[col].nunique() < 50:
#         embed[col] = 50

# a max of 50
for col in cat_names:
    embed[col] = min(50, df[col].nunique()) 

In [15]:
embed

{'bin_0': 2,
 'bin_1': 2,
 'bin_2': 2,
 'bin_3': 2,
 'bin_4': 2,
 'nom_0': 3,
 'nom_1': 6,
 'nom_2': 6,
 'nom_3': 6,
 'nom_4': 4,
 'nom_5': 50,
 'nom_6': 50,
 'nom_7': 50,
 'nom_8': 50,
 'nom_9': 50,
 'ord_0': 3,
 'ord_1': 5,
 'ord_2': 6,
 'ord_3': 15,
 'ord_4': 26,
 'ord_5': 50,
 'day': 7,
 'month': 12}

In [16]:
len(cat_names)

23

In [17]:
valid_idx=np.random.random_integers(0,600000,10000)
valid_idx

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 600000 + 1) instead
  """Entry point for launching an IPython kernel.


array([116636, 586135, 497505, 541560, ..., 583677, 579894, 536437, 124060])

In [18]:
len(valid_idx)

10000

In [19]:
# test = TabularList.from_df(test_df, path=path, cat_names=cat_names, cont_names=cont_names)#, procs=procs)

test = TabularList.from_df(test_df, path=path, cat_names=cat_names)#, cont_names=cont_names)

In [20]:
# data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
#                            .split_by_idx(list(range(590000,599999)))
#                            .label_from_df(cols=dep_var)
#                            .add_test(test)
#                            .databunch(bs=1024))

data = (TabularList.from_df(df, path=path, cat_names=cat_names, procs=procs)
                           .split_by_idx(valid_idx)
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch(bs=1024))

In [21]:
data.show_batch(rows=5)

bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0.0,0.0,0.0,T,N,Blue,Triangle,Lion,Russia,Bassoon,a8a4c2799,733d5ec74,e59dd3e27,b279e09c3,1be20a49a,1.0,Novice,Freezing,c,G,cg,7.0,12.0,0
0.0,1.0,0.0,T,N,Red,Trapezoid,Lion,Russia,Theremin,09759f3f8,7a9ef9daa,56d35c774,a8009baf5,ad3077594,3.0,Novice,Cold,c,E,DN,3.0,3.0,0
0.0,0.0,0.0,F,N,Blue,Circle,Lion,Russia,Theremin,899f15109,fef019413,afac5238e,a625ea144,3ba5f7351,1.0,Master,Freezing,a,N,RT,7.0,3.0,1
0.0,0.0,0.0,T,Y,Red,#na#,Axolotl,#na#,Bassoon,#na#,794219f89,0764632bc,f7b3f1d13,56151c280,2.0,Contributor,Freezing,m,A,vx,6.0,12.0,0
0.0,0.0,1.0,F,N,Green,Polygon,Lion,China,Bassoon,dc1470ae0,cf13fd27b,83bdea3a5,#na#,a7bbd0454,2.0,Contributor,Hot,a,Z,MF,3.0,5.0,0


In [22]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)

In [24]:
df.drop(labels='target', axis=1)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
5,0.0,NaN,1.0,T,N,Red,Triangle,Lion,China,Bassoon,...,690411ac0,05afc0f8b,2.0,Expert,Hot,b,Q,wa,3.0,4.0
6,0.0,0.0,0.0,F,N,Red,Triangle,Hamster,Costa Rica,Bassoon,...,b8e63cace,4d3766412,1.0,Grandmaster,Cold,c,R,rg,5.0,6.0
7,0.0,0.0,1.0,T,N,Red,Triangle,Axolotl,Finland,Bassoon,...,bd9643a20,a651dec43,3.0,Expert,Cold,b,Y,PS,1.0,1.0
8,0.0,0.0,0.0,F,N,Blue,Polygon,Hamster,Russia,Oboe,...,94c5fd40c,NaN,1.0,Novice,Boiling Hot,c,N,mX,6.0,3.0


In [30]:
len(data.valid_ds)

10000

In [35]:
pred_list = []
for train_idx, valid_idx in skf.split(df.drop(labels='target', axis=1),df['target']):
    print(train_idx[:10], valid_idx[:10])
    print(len(train_idx), len(valid_idx))
    data_fold = (TabularList.from_df(df, path=path, cat_names=cat_names, procs=procs)
                 .split_by_idxs(train_idx, valid_idx)
                 .label_from_df(cols=dep_var)
                 .add_test(test)
                 .databunch(bs=1024))
    print(len(data_fold.train_ds), len(data_fold.valid_ds), len(data_fold.test_ds))
    
    learn = tabular_learner(data_fold, layers=[300, 300], emb_drop=0.04, ps=[0.001, 0.01],metrics=AUROC(),
                            callback_fns=[partial(EarlyStoppingCallback, monitor='AUROC', min_delta=0.01, patience=3),
                                         ])
    learn.loss_func = nn.CrossEntropyLoss()
    
    learn.fit(10, 3e-3, wd = 0.1)
    
    pred = learn.get_preds(DatasetType.Test)
    pred_list.append(pred[0].numpy())
    break

[ 0  1  3  4  5  8  9 10 11 12] [ 2  6  7 18 28 30 35 36 44 58]
479999 120001
479999 120001 400000


epoch,train_loss,valid_loss,auroc,time


Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [ ]:


weights = [0.3, 0.7]
class_weights=torch.FloatTensor(weights)
class_weights

In [ ]:
# learn = tabular_learner(data, layers=[300,200,100], metrics=accuracy)

# learn = tabular_learner(data, layers=[300, 300],emb_szs=embed, emb_drop=0.04,
#                         ps=[0.001, 0.01], metrics=accuracy)

# Deafult Embedding Sizes

learn = tabular_learner(data, layers=[300, 300], emb_drop=0.04, ps=[0.001, 0.01], metrics=AUROC())
learn.loss_func = nn.CrossEntropyLoss(class_weights)

In [ ]:
learn.model

In [ ]:
learn.loss_func

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
data.batch_size

In [ ]:
learn.fit(2, 3e-4, wd = 0.1)

In [ ]:
learn.save('66auroc')

In [ ]:
learn.export('66leaderboard.pkl')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-04)

In [ ]:
learn.recorder.plot_losses()

## Inference

In [ ]:
learn.load('1')

In [ ]:
sample.head()

In [ ]:
row = df.iloc[0]

In [ ]:
DatasetType.Test

In [ ]:
pred = learn.get_preds(DatasetType.Test)

In [ ]:
pred[0]

In [ ]:
def get_submision(preds):
    labelled_preds = []
    pred11 = preds
    for pred in tqdm(pred11):
        labelled_preds.append(int(np.argmax(pred)))
    
    sample['target'] = labelled_preds
    
    return sample

In [ ]:
submission = get_submision(pred[0])

In [ ]:
submission.head()

In [ ]:
# submission.reset_index(drop=True)

In [ ]:
submission.to_csv('./Cat_Crent_w_66beyond.csv',index=False)

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
np.sum()